In [1]:
from DTG.classes.LLM import *
from DTG.prompts import prompt_funcs
import json, time

In [2]:
INPUT_INFO = 'NPC personality : sarcastic, rude. NPC mood : sarcastic, annoyed. NPC name : Renzov'

In [8]:
import json
import json
def get_dialogue_json(input_info):
    t_0 = time.time()
    starting_time = time.time()
    performance_dict = {
        'tags' : 0,
        'abstract' : 0,
        'endings' : 0,
        'tree' : 0,
        'formatted_json' : 0
    }
    llm = LLM()
    print(f'Generating tags.')
    tags_by_llm = llm.send_message_for_code(prompt_funcs.tag_prompt(input_info), model='gpt-4o-mini')
    print(f'Tags generated in {time.time() - starting_time} seconds.')
    performance_dict['tags'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating abstract.')
    abstract = llm.send_message_for_code(prompt_funcs.abstract_prompt(tags_by_llm, input_info))
    print(f'Abstract generated in {time.time() - starting_time} seconds.')
    performance_dict['abstract'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating possible endings.')
    endings_by_llm = llm.send_message_for_code(prompt_funcs.possible_ending_prompt(abstract), model='gpt-4o-mini')
    print(f'Possible endings generated in {time.time() - starting_time} seconds.')
    performance_dict['endings'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating dialogue tree.')
    tree_by_llm = llm.send_message_for_code(prompt_funcs.dialogue_prompt(abstract, endings_by_llm, tags_by_llm), model='o1-preview')
    print(f'Dialogue tree generated in {time.time() - starting_time} seconds.')
    performance_dict['tree'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Formatting JSON.')
    formatted_tree = llm.send_message(prompt_funcs.format_prompt(tree_by_llm), model='gpt-4o-mini')
    print(f'JSON formatted in {time.time() - starting_time} seconds.')
    performance_dict['formatted_json'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Total time taken : {time.time() - t_0} seconds.')
    print(f'Performance dict : {performance_dict}')
    formatted_tree_json = json.loads(formatted_tree)
    return formatted_tree_json


In [4]:
tree = get_dialogue_json(INPUT_INFO)

Generating tags.
Tags generated in 4.112805128097534 seconds.
Generating abstract.
Abstract generated in 4.076787948608398 seconds.
Generating possible endings.
Possible endings generated in 2.470720052719116 seconds.
Generating dialogue tree.
Dialogue tree generated in 52.51952004432678 seconds.
Formatting JSON.
JSON formatted in 15.457004070281982 seconds.
Total time taken : 78.63762497901917 seconds.
Performance dict : {'tags': 4.112875938415527, 'abstract': 4.077003002166748, 'endings': 2.470893144607544, 'tree': 52.51974606513977, 'formatted_json': 15.457102060317993}


In [6]:
json_tree = json.loads(tree)

In [7]:
json_tree

{'nodes': [{'id': 'node1',
   'npc_dialogue': "Oh look, if it isn't our esteemed manager gracing us with their presence. To what do we owe this dubious pleasure?",
   'tag_of_node': '#greeting #sarcastic #rude #unfriendly #annoyed',
   'player_dialogue_choices': [{'player_dialogue_choice': "Just checking in to see how you're finding the garden.",
     'next_node': 'node2a'},
    {'player_dialogue_choice': 'I sensed some dissatisfaction; is there anything I can help with?',
     'next_node': 'node2b'},
    {'player_dialogue_choice': 'Is there a problem, Renzov?',
     'next_node': 'node2c'}]},
  {'id': 'node2a',
   'npc_dialogue': "Finding? Oh, I'm not lost, but this garden seems to be. Was the 'crumbling ruins' look intentional or just a happy accident?",
   'tag_of_node': '#question #critical #dissatisfied #sarcastic #rude',
   'player_dialogue_choices': [{'player_dialogue_choice': "I appreciate your honesty. I'll see to the garden's improvements.",
     'next_node': 'node3a1'},
    {